Install

In [ ]:
#!pip install geoplot
#!pip install geojsonio
#!pip install up42-py
#!pip install ipyleaflet

Import

In [ ]:
import geopandas as gpd
import folium
import up42

### Explore test site 

read road alignment file

In [ ]:
road_alignment = gpd.read_file('./road_alignment.geojson')

In [ ]:
road_alignment.crs

In [ ]:
# Read the GeoJSON file
road_alignment = gpd.read_file('./road_alignment.geojson')

# Transform coordinate system
road_alignment = road_alignment.to_crs('EPSG:4326')  # Change to WGS84 if needed

# Create a buffer around the line geometry
buffer_distance = 0.001  
road_alignment_buffer = road_alignment.buffer(buffer_distance)

# Convert GeoDataFrame to GeoJSON object
geojson_data_buffer = road_alignment_buffer.to_json()
geojson_data_line = road_alignment.to_json()

# Create a base map
m = folium.Map(location=[road_alignment.geometry.centroid.y.mean(), road_alignment.geometry.centroid.x.mean()], zoom_start=10)

# Add the buffer layer to the map
folium.GeoJson(geojson_data_buffer, name='Buffer').add_to(m)

# Add the original line layer to the map
folium.GeoJson(geojson_data_line, name='Road Alignment').add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Show the map
m


In [ ]:
# Convert the buffer to a GeoDataFrame
road_alignment_buffer_gdf = gpd.GeoDataFrame(geometry=road_alignment_buffer, crs='EPSG:4326')

# Save the buffer as a GeoJSON file
aoi = 'road_alignment_buffer.geojson'
road_alignment_buffer_gdf.to_file(aoi, driver='GeoJSON')

### UP42 catalog data exploration

In [ ]:
up42.authenticate(cfg_file="credentials.json")

In [ ]:
aoi = up42.read_vector_file(aoi, as_dataframe=False)

In [ ]:
# Identify the right data product for your use-case
catalog = up42.initialize_catalog()
data_product_id = catalog.get_data_products(basic=True).get("Sentinel-2").get("data_products").get("Level-2A")
data_products = catalog.get_data_products(basic=True)

In [ ]:
# Search and select the right scene for your use-case
search_results = catalog.search(search_parameters=catalog.construct_search_parameters(
    geometry=aoi,
    start_date="2023-05-01", end_date="2023-09-30",
    collections=[data_products.get("Sentinel-2").get("collection")],
    max_cloudcover=10, limit=10))

In [ ]:
# Place and track the order of your selected scene
order_parameters = catalog.construct_order_parameters(
    data_product_id=data_product_id, image_id=search_results.id[0])
catalog.estimate_order(order_parameters)
order = catalog.place_order(order_parameters, track_status=True)

In [ ]:
# Stream cloud-native files directly for your use case
asset = up42.initialize_order(order_id=order.order_id).get_assets()[0]
stac_items = asset.stac_items
asset.get_stac_asset_url(stac_asset=stac_items[0].assets.get("b02.tiff"))